In [1]:
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.9 MB/s eta 0:00:00


In [3]:
!pip install lancedb --q
!pip install pypdf --q
!pip install -qqq langchain --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install transformers -q

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

loader = PyPDFLoader("/content/Aalborghuskollegiet A-Z (English).pdf")

docs = loader.load()
len(docs)

11

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

49

In [6]:
import lancedb
from langchain.vectorstores import LanceDB
from langchain.embeddings import HuggingFaceEmbeddings


uri = "/content/data/AABH_db"
db = lancedb.connect(uri)

In [7]:
embeddings = HuggingFaceEmbeddings()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
table = db.create_table(
    "AABH",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

docsearch = LanceDB.from_documents(texts, embeddings, connection=table)

In [11]:
retriever = docsearch.as_retriever(search_kwargs={'k': 2})

In [12]:
retriever

VectorStoreRetriever(tags=['LanceDB', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.lancedb.LanceDB object at 0x7bd381c4e5f0>, search_kwargs={'k': 2})

In [13]:
texts[0].page_content

'Aalborghuskollegiet A -Z \n(English)  \n \n \n \nIntro \nThe property:  \nAalborghus Kollegiet is located on a sideway to Sohngårdsholmsvej, where busses goes  to \nand from the city center and AAU. The immediate closeness to the city center and AAU \nmakes the area attractive to students. Few minutes walk from the dorm there is grocery \nstores, pharmacy, pizza place, fitness etc.  \nThe Aalborghus collegium is a dormitory in two floors with 107 rooms and 4 apartments'

In [14]:
result = table.search(embeddings.embed_query(texts[0].page_content)).limit(2).to_list()

In [15]:
result[0].keys()

dict_keys(['vector', 'text', 'id', '_distance'])

In [30]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [31]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [33]:
repo_id = "mistralai/Mistral-7B-v0.1"

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0.2, "max_new_tokens":50})

In [27]:
# Example user query

user_query = "Tell me where i can find the cinema"

retrieved_texts = table.search(embeddings.embed_query(user_query)).limit(5).to_list()

context = " ".join([result['text'] for result in retrieved_texts])

In [37]:
full_prompt = f"""Based on the following information: {context} \n
Answer the question: {user_query}"""
response = llm.generate([full_prompt], max_length=150)

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: zCFIrj2lfTSfNkE-b4omA)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [35]:
response

NameError: name 'response' is not defined